# Part 1 - Deal with the Wikipedia Table

Import modules first.

In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Request data from the Wikipeida page. Use BeautifulSoup to get the html content, and apply __soup.find__ to get the part corresponding to the target table.

In [12]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
soup = BeautifulSoup(req.text, "html5lib")
data = soup.find(class_="wikitable sortable")

Define a new dataframe __neighborhoods__ with the desired columns.

In [13]:
column_names = ['Postal Code', 'Borough', 'Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)

Extract data of the table row by row. For each row, I record data into dataframe only if the corresponding borough is assigned.

When the loop reaches the end of the table, __the postal code on next line__ is no longer a string of 3 characters, indicating the end of data recording.

In [14]:
header = data.findChild("tr")
dataline = header.find_next('tr')

while True:
    post0 = dataline.find_next('td')
    post = post0.get_text(strip=True) 
    
    bor0 = post0.find_next('td')
    bor = bor0.get_text(strip=True)
    # skip the data whose borough is not assigned     
    if bor != 'Not assigned':
        neigh0 = bor0.find_next('td')
        neigh = neigh0.get_text(strip=True)
            
        neighborhoods = neighborhoods.append({'Postal Code': post,
                                              'Borough': bor,
                                              'Neighborhood': neigh}, ignore_index=True)
    dataline = dataline.find_next('tr') 
    post_check = dataline.find_next('td').get_text(strip=True)
    # reach the end of the table
    if len(post_check) != 3:
        break

The first ten rows of the dataframe __neighborhoods__:

In [15]:
neighborhoods.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


The row count of the dataframe __neighborhoods__:

In [16]:
print('The number of rows in the dataframe: ',neighborhoods.shape[0])

The number of rows in the dataframe:  103


# Part 2 - Add Latitude and Longitude 

Read the latitude and the longitude values from the csv file.

In [17]:
latlon = pd.read_csv('http://cocl.us/Geospatial_data')
#latlon.sort_values(by='Postal Code', inplace=True)
#latlon.reset_index(drop=True, inplace=True)
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sort the __neighborhoods__ dataframe from Part 1 by postal code.

In [18]:
neighborhoods.sort_values(by='Postal Code', inplace=True)
neighborhoods.reset_index(drop=True, inplace=True)

Concatenate 2 dataframes. Also record the postal code information in __latlon__ and rename it as *Postal Code1* column.

In [19]:
neighborhoods[['Postal Code1', 'Latitude', 'Longitude']] = latlon[latlon.columns.tolist()]
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Postal Code1,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


Check if Postal Code of the 2 dataframes are identical. 

The output is True. The 2 dataframes are properly concatenated.

In [20]:
# check if Postal Code of the 2 dataframes are identical
sum(neighborhoods['Postal Code'] == neighborhoods['Postal Code1']) == neighborhoods.shape[0]

True

Drop the *Postal Code1* column. Now we have the desired dataframe.

In [21]:
neighborhoods.drop(columns=['Postal Code1'], inplace=True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 - Explore and Cluster the Neighborhoods in Toronto

Target: boroughs whose name includes __Toronto__

In [22]:
nb_tor = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
nb_tor.reset_index(drop=True, inplace=True)
print('There are', nb_tor.shape[0], 'target sets of neighborhoods in Toronto.')
nb_tor.head()

There are 39 target sets of neighborhoods in Toronto.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Here I use the same function *getNearbyVenues, return_most_common_venues* from the previous lab: __Lab - Segmenting and Clustering Neighborhoods in New York City__.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Import modules.

In [41]:
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
# The code was removed by Watson Studio for sharing.

In [26]:
tor_venues = getNearbyVenues(names=nb_tor['Neighborhood'], latitudes=nb_tor['Latitude'], longitudes=nb_tor['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [27]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


### Do the same analysis as in __Lab - Segmenting and Clustering Neighborhoods in New York City__.

#### 1. construct a dataframe to record the common venues (here top 5 common venues) for each neighborhoods.

In [37]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Seafood Restaurant,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bar,Intersection
2,Business reply mail Processing Centre,Yoga Studio,Skate Park,Fast Food Restaurant,Light Rail Station,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Burger Joint


#### 2. Run *k*-means to cluster the neighborhoods into 5 clusters, and then put all the desired information into a dataframe.

In [38]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = nb_tor
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Women's Store,Cupcake Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Pizza Place,Ice Cream Shop,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Ethiopian Restaurant,Eastern European Restaurant


#### 3. Visualize the clusters.

In [72]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
colors_array = cm.rainbow([0, 0.15, 0.3, 0.7, 1.0])
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 4. Examine clusters

Cluster 1

In [62]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[2,3,4] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Women's Store,Cupcake Shop
1,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore
2,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Pizza Place,Ice Cream Shop,Fish & Chips Shop
3,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery
5,Davisville North,43.712751,-79.390197,0,Park,Hotel,Gym,Sandwich Place,Breakfast Spot
6,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Miscellaneous Shop,Seafood Restaurant
7,Davisville,43.704324,-79.388790,0,Pizza Place,Dessert Shop,Sandwich Place,Gym,Coffee Shop
9,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,0,Coffee Shop,Pub,Supermarket,Fried Chicken Joint,Sports Bar
11,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Restaurant,Bakery,Pizza Place,Chinese Restaurant
12,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio


Cluster 2

In [64]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[2,3,4] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,"Moore Park, Summerhill East",43.689574,-79.38316,1,Playground,Women's Store,Cupcake Shop,Electronics Store,Eastern European Restaurant


Cluster 3

In [65]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2,3,4] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Rosedale,43.679563,-79.377529,2,Park,Trail,Playground,Creperie,Donut Shop
23,Forest Hill North & West,43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop


Cluster 4

In [66]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[2,3,4] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Lawrence Park,43.72802,-79.38879,3,Park,Bus Line,Swim School,Ethiopian Restaurant,Eastern European Restaurant


Cluster 5

In [67]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[2,3,4] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Roselawn,43.711695,-79.416936,4,Garden,Music Venue,Home Service,Women's Store,Dance Studio


### Observation:

Cluster 1 - Food, Cafe, Bar

Cluster 2 - Playground

Cluster 3 - Park, Trails

Cluster 4 - Park, Bus services

Cluster 5 - Garden, Music